In [ ]:
import json

In [ ]:
import redis

In [ ]:
pdata = h5file.pdata

In [ ]:
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

In [46]:
def find_pa(x:int):
    #父节点为本身，直接返回。
    if parent[x]==x:
        return x
    # 类似记忆化储存.
    parent[x]=find_pa(parent[x])
    return parent[x]


def union_(x:int, y:int):
    x_root=find_pa(x)
    y_root=find_pa(y)
    if x_root==y_root:
        # 两节点父节点是同一个，即两节点在同一个集合中，无需合并
        return
    else:
        # 合并操作，可让其中任意一个父节点的父节点为对方的父节点；
        # parent[x_root]=y_root  
        parent[y_root] = x_root
        return

parent = {}
for i in range(1,10):
    parent[i] = i
    
union_(8,9)
union_(4,8)
union_(5,6)
union_(2,4)
union_(1,3)
union_(1,2)
parent

In [ ]:
parent = {}
rank = {}
for i in range(1,10):
    parent[i] = i
    rank[i] = 1

merge(8,9)
merge(4,8)
merge(5,6)
merge(2,4)
merge(1,3)
merge(1,2)

parent

In [100]:
def find(x):
    #父节点为本身，直接返回。
    if parent[x]==x:
        return x
    # 类似记忆化储存.
    parent[x]=find(parent[x])
    return parent[x]


def merge(i, j):
    # 先找到两个根节点
    x = find(i)
    y = find(j) 
    if rank[x] <= rank[y]:
        parent[x] = y
    else:
        parent[y] = x
    if rank[x] == rank[y] and x != y:
        # 如果深度相同且根节点不同，则新的根节点的深度+1
        rank[y] += 1;                   

        
def mergeColumnAndTitle(id_set, column_dic, title_dic):
    """
    路径优化版本的并查集，接收三个参数：
    id_set:全部id的set，由pdata.id获得
    （注：这里为了加速计算，直接接收id_set，实际应该检查以下id_set ?= column_dic.values+title_dic.values）
    column_dic:全部有column的字典，结构为{'doi':[属于这个doi的所有id...]}
    title_dic:全部没有column的字典，结构为{'未定':[title相似的所有id...]}
    返回：{'new_id':[所有重复的id...]}
    """
    # 初始化
    parent = {}
    rank = {}
    for id_ in id_set:
        parent[id_] = id_
        rank[id_] = 1
        
    # 合并，（1，2，3）拆分为（1，2）（1，3）来合并
    for key,value in column_dic.items():
        if len(value) > 1:
            for dup_id in value[1:]:
                merge(value[0],dup_id)

    for key,value in title_dic.items():
        if len(value) > 1:
            for dup_id in value[1:]:
                merge(value[0],dup_id)
            
    # 找出重复项(按根节点分类)
    dup_dic = {}
    for id_ in id_set:
        root_id = find(id_)
        if root_id not in dup_dic:
            dup_dic[root_id] = [id_]
        else:
            dup_dic[root_id].append(id_)
    
#     # 创建新id
#     final_dup_dic = {}
#     for dup_id_list in dup_dic.values():
#         new_id = 产生新id的函数
#         final_dup_dic[new_id] = dup_id_list
#     return final_dup_dic
    return dup_dic

In [101]:
#测试

In [11]:
# 全部id，从pdata中获取
id_set = set()
for i in range(1,20):
    id_set.add('id'+str(i))

# 有doi的所有有column的字典
column_dic = {'doi1':['id1','id2','id3','id4'],\
             'doi2':['id5'],\
             'doi3':['id6','id7'],\
             'doi4':['id8'],\
             'doi5':['id9']}

In [12]:
# 无doi的，只会和id1、5、6、8、9匹配
"""
出现18、8、9的情况是，id8和id9的doi不同（不重复）但他们标题相似，且都与id18相似
这时无法判断（虽然后续可以再通过其他字段来判断，但是暂时不考虑）18到底是和8还是9重复，我们在这里取第一个8作为他的重复项
所以在索引为len1～len2之间匹配（指计算fuzz距离）时，只要匹配到1个相似就应该停止匹配
不仅可以加速而且可以避免出错（把8、9匹配成了重复，然而他们本身不重复）
注：len1为doi为空 的哈希桶的长度，len2为去重后distinct 的哈希桶的长度
"""
title_dic = {'hash_code1_1':['id10','id11','id12','id1'],\
            'hash_code1_2':['id13'],\
            'hash_code2_1':['id14','id15','id16'],\
            'hash_code3_1':['id17'],\
            'hash_code4_1':['id18','id8','id9'],\
            'hash_code5_1':['id19','id5']}

In [13]:
from merge import mergeColumnAndTitle
mergeColumnAndTitle(id_set, column_dic, title_dic)

{1442295841757995008: ['id13'],
 1442295841757995009: ['id3', 'id2', 'id4', 'id10', 'id1', 'id11', 'id12'],
 1442295841757995010: ['id17'],
 1442295841757995011: ['id8', 'id18', 'id9'],
 1442295841757995012: ['id15', 'id16', 'id14'],
 1442295841757995013: ['id19', 'id5'],
 1442295841757995014: ['id7', 'id6']}

In [105]:
# 生成ID
from generate_id import IdWorker
worker = IdWorker(1,2)

In [111]:
worker.get_id()

1442052586789281792

In [24]:
pdata1 = pd.read_hdf('data/11_2021.h5')
pdata1.head()

,id,title,language,identifier_doi,source_file
0,21134952343,积极发挥优势 增强发展后劲,,,111
1,21134961091,苏州高新区：文旅产业同样“乘风破浪”,,10.28870/n.cnki.nxhmr.2021.000920,111
2,21134965150,庆阳市第四届人民代表大会第六次会议主席团常务主席名单,,,111
3,21134968931,2020年全国海关减（退）税超过2600亿元,,10.28264/n.cnki.ngjcd.2021.000155,111
4,21134969660,河北省调整相关区域疫情风险等级,,10.28326/n.cnki.nhbrb.2021.000351,111


In [25]:
len(pdata1)

771280

In [26]:
(pdata1.identifier_doi == '').sum()

224317

In [27]:
(pdata1.identifier_doi.value_counts() > 1).sum()

1

In [ ]:
(pdata.language == 'ZH').count()
(pdata.language == 'ZH').sum()
pdata.loc[pdata.language == 'ZH', 'language']

In [ ]:
pdata1.identifier_doi.unique()

In [ ]:
pdata1[pdata1.identifier_doi == '10.1016/j.scitotenv.2020.141974']

In [ ]:
(pdata1[pdata1.language == 'EN'].identifier_doi.value_counts()>2).sum()